In [ ]:
from IPython.display import display, Markdown
import snakemd

import fitfile
from garmindb import ConfigManager
from garmindb.garmindb import GarminDb, Attributes, ActivitiesDb, Activities, ActivityLaps, ActivityRecords
from idbutils.list_and_dict import list_not_none


db_params_dict = ConfigManager.get_db_params()
garmin_db = GarminDb(db_params_dict)
garmin_act_db = ActivitiesDb(db_params_dict)
measurement_system = Attributes.measurements_type(garmin_db)
unit_strings = fitfile.units.unit_strings[measurement_system]
distance_units = unit_strings[fitfile.units.UnitTypes.distance_long]

def __report_sport(sport_col, sport):
    records = Activities.row_count(garmin_act_db, sport_col, sport)
    if records > 0:
        sport_title = sport.title().replace('_', ' ')
        total_distance = Activities.get_col_sum_for_value(garmin_act_db, Activities.distance, sport_col, sport)
        if total_distance is None:
            total_distance = 0
            average_distance = 0
        else:
            average_distance = total_distance / records
        return [sport_title, records, total_distance, average_distance]

doc = snakemd.new_doc("activities")

doc.add_header("Activities Statistics")
doc.add_paragraph("Analysis of all activities in the database.")

doc.add_table(
    ['Type', 'Count'],
    [
        ["Total activities", Activities.row_count(garmin_act_db)],
        ["Total Lap records", ActivityLaps.row_count(garmin_act_db)],
        ["Activity records", ActivityRecords.row_count(garmin_act_db)],
        ["Fitness activities", Activities.row_count(garmin_act_db, Activities.type, 'fitness')],
        ["Recreation activities", Activities.row_count(garmin_act_db, Activities.type, 'recreation')]
    ])

years = Activities.get_years(garmin_act_db)
years.sort()
doc.add_paragraph(f"Years with activities: {len(years)}: {years}")
sports = list_not_none(Activities.get_col_distinct(garmin_act_db, Activities.sport))
doc.add_paragraph(f"Sports: {', '.join(sports)}")
sub_sports = list_not_none(Activities.get_col_distinct(garmin_act_db, Activities.sub_sport))
doc.add_paragraph(f"SubSports: {', '.join(sub_sports)}")

sports_stats = []
for sport_name in [sport.name for sport in fitfile.Sport]:
    sport_stat = __report_sport(Activities.sport, sport_name)
    if sport_stat:
        sports_stats.append(sport_stat)
doc.add_header("Sport Type Statistics", 3)
doc.add_table(['Sport', 'Total Activities', f'Total Distance ({distance_units})', f"Average Distance ({distance_units})"], sports_stats)

activities = Activities.get_latest(garmin_act_db, 10)
rows = [[activity.activity_id, activity.name, activity.type, activity.sport, activity.elapsed_time, activity.calories] for activity in activities]
doc.add_header("Last Ten Activities", 3)
doc.add_table(['Id', 'Name', 'Type', 'Sport', 'Elapsed Time', 'Calories'], rows)

display(Markdown(str(doc)))